In [1]:
from bokeh.io import gridplot, output_notebook, output_file, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [2]:
import numpy as np
import os
import cPickle as pkl

import time
import datetime

import pandas as pd
import scipy.io
import copy

def get_timekey(item):
    return item.time


In [3]:
# Import functions for getting event info needed to parse MW trials:
from parse_mw_events import get_bar_events, get_stimulus_events


In [4]:
# Set datafile paths in opts:
prepend = '/' #'/Users/julianarhee'
source_dir = os.path.join(prepend,'nas/volume1/2photon/RESDATA')  # options.source_dir

session = '20161222_JR030W'

experiment = 'gratings2'
#parser.add_option("--experiment", action="append", dest="experiment_list")

stimtype = 'grating' #options.stimtype #'grating'
#mask = False #options.mask # False
#long_trials = False #options.long_trials #True
no_ard = True #options.no_ard

In [5]:
# Look in child dir (of source_dir) to find mw_data paths:
data_dir = os.path.join(source_dir, session, experiment)
mw_dir = os.path.join(data_dir, 'mw_data')
mwfiles = os.listdir(mw_dir)
mwfiles = [m for m in mwfiles if m.endswith('.mwk')]

# TODO:  adjust path-setting to allow for multiple reps of the same experiment


In [6]:
# Cycle through MW files (1 file per experiment rep):
all_dfns = []
for mwfile in mwfiles:
    fn_base = mwfile[:-4]
    mw_fn = fn_base+'.mwk'
    mw_dfn = os.path.join(mw_dir, mw_fn)
    all_dfns.append(mw_dfn)
    if not no_ard:
        ard_fn = fn_base+'.txt'
        ard_dfn = os.path.join(mw_dir, ard_fn)
        all_dfns.append(ard_dfn)
mw_dfns = [f for f in all_dfns if f.endswith('.mwk')]
ar_dfns = [f for f in all_dfns if f.endswith('.txt')]


print "MW file: ", mw_dfns

MW file:  ['/nas/volume1/2photon/RESDATA/20161222_JR030W/gratings2/mw_data/20161222_JR030W_gratings_20reps_run2.mwk']


In [7]:
# Get MW events
didx = 0
if stimtype=='bar':
    pixelevents, stimevents, trigger_times, session_info = get_bar_events(mw_dfns[didx])
else:
    pixelevents, stimevents, trialevents, trigger_times, session_info = get_stimulus_events(mw_dfns[0], stimtype=stimtype)
    

****************************************************************
Parsing file
/nas/volume1/2photon/RESDATA/20161222_JR030W/gratings2/mw_data/20161222_JR030W_gratings_20reps_run2.mwk... 
Found 1 start events in session.
Bounds:  [[191435624202, 193765802665]]
bound ID: 0 2330.178463 sec
****************************************************************
[[191435624202, 193765802665]]
................................................................
SECTION 0
................................................................
Found > 1 name for frame-trigger:
Choose:  ['frame_trigger_timeout', 'frame_trigger_on', 'trigger_received', 'trigger_active', 'frame_trigger', 'frame_triggered']
Hint: RSVP could be FrameTrigger, otherwise frame_trigger.
Type var name to use: frame_trigger
first SI-trigger ON event received:  Event[code=46, name=frame_trigger, time=191454011428, value=0]
first SI-trigger OFF event received:  Event[code=46, name=frame_trigger, time=191536123443, value=1]
Duration of first 

In [8]:
# For EACH boundary found for a given datafile (dfn), make sure all the events are concatenated together:
print "================================================================"
print "MW parsing summary:"
print "================================================================"
if len(pixelevents) > 1:
    pixelevents = [item for sublist in pixelevents for item in sublist]
    pixelevents = list(set(pixelevents))
    pixelevents.sort(key=operator.itemgetter(1))
else:
    pixelevents = pixelevents[0]
print "Found %i pixel clock events." % len(pixelevents)

# Check that all possible pixel vals are used (otherwise, pix-clock may be missing input):
n_codes = set([i.value[-1]['bit_code'] for i in pixelevents])
if len(n_codes)<16:
    print "Check pixel clock -- missing bit values..."


if len(stimevents) > 1:
    stimevents = [item for sublist in stimevents for item in sublist]
    stimevents = list(set(stimevents))
    stimevents.sort(key=operator.itemgetter(1))
else:
    stimevents = stimevents[0]
print "Found %i stimulus on events." % len(stimevents)

if len(trialevents) > 1:
    trialevents = [item for sublist in trialevents for item in sublist]
    trialevents = list(set(trialevents))
    trialevents.sort(key=operator.itemgetter(1))
else:
    trialevents = trialevents[0]
print "Found %i trial epoch (stim ON + ITI) events." % len(trialevents)


if len(trigger_times) > 1:
    trigger_times = [item for sublist in trigger_times for item in sublist]
    trigger_times = list(set(trigger_times))
    trigger_times.sort(key=operator.itemgetter(1))
else:
    trigger_times = trigger_times[0]
print "Found %i runs (i.e., trigger boundary events)." % len(trigger_times)

if len(session_info) > 1:
    session_info = [item for sublist in session_info for item in sublist]
    session_info = list(set(trigger_times))
    session_info.sort(key=operator.itemgetter(1))
else:
    session_info = session_info[0]
#print "N sessions in file: .", len(session_info)
print session_info


MW parsing summary:
Found 43309 pixel clock events.
Found 699 stimulus on events.
Found 1398 trial epoch (stim ON + ITI) events.
Found 25 runs (i.e., trigger boundary events).
{'tboundary': [191435624202, 193765802665], 'stimduration': 1000, 'stimsize': 50, 'ITI': 2000}


In [10]:
# on FLASH protocols, first real iamge event is 41
print "Found %i trials, corresponding to %i TIFFs." % (len(stimevents), len(trigger_times))
refresh_rate = 60

# Creat trial-dicts for each trial in run, in format for NDA database:

if stimtype=='bar':
    
    nexpected_pixelevents = int(round((1/session_info['target_freq']) * session_info['ncycles'] * refresh_rate * len(trigger_times)))
    print "Expected %i pixel events, missing %i pevs." % (nexpected_pixelevents, nexpected_pixelevents-len(pixelevents))

    stimnames = ['left', 'right', 'top', 'bottom']

    # GET TRIAL INFO FOR DB:
    trial_list = [(stimevents[k].ordernum, k) for k in stimevents.keys()]
    trial_list.sort(key=lambda x: x[0]) 
    trial = dict((i+1, dict()) for i in range(len(stimevents)))

    for trialidx,mvtrial in enumerate(trial_list):
        mvname = mvtrial[1]
        trialnum = trialidx + 1
        trial[trialnum]['start_time_ms'] = round(stimevents[mvname].states[0][0]/1E3)
        trial[trialnum]['end_time_ms'] = round(stimevents[mvname].states[-1][0]/1E3)
        stimname = [i for i in stimnames if i in mvname][0]
        stimsize = session_info['target_freq']
        trial[trialnum]['stimuli'] = {'stimulus': stimname, 'position': stimevents[mvname].states[0][1], 'scale': stimsize}
        trial[trialnum]['stim_on_times'] = round(stimevents[mvname].states[0][0]/1E3)
        trial[trialnum]['stim_off_times'] = round(stimevents[mvname].states[-1][0]/1E3)

else:
    
    ntrials = len(stimevents)
    itis = sorted(trialevents[1::2], key=get_timekey) # get_stimulus_events() returns alternating list of [trial1-stimonset, trial2-iti, trial2-stimonset, trial2-iti, etc...]
    
    # Get dynamic-grating bicode events:
    dynamic_stim_bitcodes = []
    for stim,iti in zip(sorted(stimevents, key=get_timekey), sorted(itis, key=get_timekey)):
        # For each trial, store all associated stimulus-bitcode events (including the 1st stim-onset) as a list of
        # display-update events related to that trial:
        current_bitcode_evs = [p for p in pixelevents if p.time>=stim.time and p.time<iti.time]
        dynamic_stim_bitcodes.append(current_bitcode_evs)
    
    # Roughly calculate how many pixel-clock events there should be. For static images, there should be 1 bitcode-event per trial.
    # For drifting gratings, on a 60Hz monitor, there should be 60-61 bitcode-events per trial.
    nexpected_pixelevents = (ntrials * (session_info['stimduration']/1E3) * refresh_rate) + ntrials
    nbitcode_events = sum([len(tr) for tr in dynamic_stim_bitcodes]) + len(itis)

    print "Expected %i pixel events, missing %i pevs." % (nexpected_pixelevents, nexpected_pixelevents-nbitcode_events)

    # GET TRIAL INFO FOR DB:
    trial = dict((i+1, dict()) for i in range(len(stimevents)))
    stimevents = sorted(stimevents, key=get_timekey)
    trialevents = sorted(trialevents, key=get_timekey)
    run_start_time = trialevents[0].time
    for trialidx,(stim,iti) in enumerate(zip(sorted(stimevents, key=get_timekey), sorted(itis, key=get_timekey))):
        trialnum = trialidx + 1
        # blankidx = trialidx*2 + 1
        trial[trialnum]['start_time_ms'] = round(stim.time/1E3)
        trial[trialnum]['end_time_ms'] = round((iti.time + session_info['ITI']))
        if stimtype=='grating':
            ori = stim.value[1]['rotation']
            sf = round(stim.value[1]['frequency'], 2)
            stimname = 'grating-ori-%i-sf-%f' % (ori, sf)
            stimpos = [stim.value[1]['xoffset'], stim.value[1]['yoffset']]
        else:
            # TODO:  fill this out with the appropriate variable tags for RSVP images
            stimname = ''
            stimpos = ''
        stimsize = stim.value[1]['height']
        trial[trialnum]['stimuli'] = {'stimulus': stimname, 'position': stimpos, 'scale': stimsize}
        trial[trialnum]['stim_on_times'] = round((stim.time - run_start_time)/1E3)
        trial[trialnum]['stim_off_times'] = round((iti.time - run_start_time)/1E3)


Found 699 trials, corresponding to 25 TIFFs.
Expected 42639 pixel events, missing -662 pevs.


In [11]:
trial[1]

# save trial info as pkl for easyloading: 
trialinfo_fn = 'trial_info.pkl'
with open(os.path.join(data_dir, 'mw_data', trialinfo_fn), 'wb') as f:
    pkl.dump(trial, f, protocol=pkl.HIGHEST_PROTOCOL)
    f.close()
    
trial[1]


{'end_time_ms': 191455053700.0,
 'start_time_ms': 191454050.0,
 'stim_off_times': 1002.0,
 'stim_on_times': 0.0,
 'stimuli': {'position': [0, 0],
  'scale': 50,
  'stimulus': 'grating-ori-45-sf-0.500000'}}

In [12]:
print "Each TIFF duration is about (sec): "
for idx,t in enumerate(trigger_times):
    print idx, (t[1] - t[0])/1E6


Each TIFF duration is about (sec): 
0 82.112015
1 82.15186
2 82.168049
3 82.168129
4 82.167975
5 82.167923
6 82.168071
7 82.167944
8 82.167926
9 82.175886
10 82.168078
11 82.17598
12 82.167958
13 82.167971
14 82.167966
15 82.167976
16 82.168044
17 82.159939
18 82.160209
19 82.168156
20 82.176141
21 82.16004
22 82.144199
23 82.168015
24 82.168006


In [13]:
# Create "pydict" to store all MW stimulus/trial info in matlab-accessible format for GUI:
if stimtype=='bar':
    pydict = dict()
    print "Offset between first MW stimulus-display-update event and first SI frame-trigger:"
    for ridx,run in enumerate(stimevents.keys()):
        pydict[run] ={'time': [i[0] for i in stimevents[run].states],\
                    'pos': stimevents[run].vals,\
                    'idxs': stimevents[run].idxs,\
                    'ordernum': stimevents[run].ordernum,\
                    'MWdur': (stimevents[run].states[-1][0] - stimevents[run].states[0][0]) / 1E6,\
                    'offset': stimevents[run].states[0][0] - stimevents[run].triggers[0],\
                    'MWtriggertimes': stimevents[run].triggers}
        print "run %i: %s ms" % (ridx+1, str(pydict[run]['offset']/1E3))
        
elif stimtype == 'image':
    # TODO:  need to test this, only debugged with GRATINGS.
    all_ims = [i.value[1]['name'] for i in ievs]
    image_ids = sorted(list(set(all_ims)))

    mw_times = np.array([i.time for i in tevs]) #np.array([i[0] for i in pevs])
    mw_codes = []
    if mask is True:
        nexpected_imvalues = 4
    else:
        nexpected_imvalues = 3

    for i in tevs:
        if len(i.value)==nexpected_imvalues:
            stim_name = i.value[1]['name']
            stim_idx = [iidx for iidx,image in enumerate(image_ids) if image==stim_name][0]+1
        else:
            stim_idx = 0
        mw_codes.append(stim_idx)
    mw_codes = np.array(mw_codes)

elif stimtype == 'grating':
    
    stimevents = sorted(stimevents, key=lambda e: e.time)
    trialevents = sorted(trialevents, key=lambda e: e.time)
    
    all_combos = [(i.value[1]['rotation'], round(i.value[1]['frequency'],1)) for i in stimevents]
    image_ids = sorted(list(set(all_combos))) # should be 35 for gratings -- sorted by orientation (7) x SF (5)


In [14]:
parse_trials = False
if stimtype != 'bar':
    #t_mw_intervals = np.diff(mw_times)
    parse_trials = True


In [15]:
# Check stimulus durations:
print len(stimevents)
iti_events = trialevents[1::2]
print len(iti_events)

stim_durs = []
off_syncs = []
for idx,(stim,iti) in enumerate(zip(stimevents, iti_events)):
    stim_durs.append(iti.time - stim.time)
    if (iti.time - stim.time)<0:
        off_syncs.append(idx)
print len(off_syncs)

# PLOT stim durations:
print "N stim ONs:", len(stim_durs)
print "min:", min(stim_durs)
print "max:", max(stim_durs)
print len([i for i,v in enumerate(stim_durs) if v>1100000])
p1 = figure(title="Stim ON durations (ms)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")
hist, edges = np.histogram(np.array(stim_durs)/1E6, density=True, bins=100)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

699
699
0
N stim ONs: 699
min: 1001499
max: 1018253
0


In [16]:
if parse_trials is True:
    mw_codes_by_file = []
    mw_times_by_file = []
    mw_trials_by_file = []
    offsets_by_file = []
    runs = dict()
    for idx,triggers in enumerate(trigger_times):
        curr_trialevents = [i for i in trialevents if i.time<=triggers[1] and i.time>=triggers[0]]
        
        # Get TIME for each stimulus trial start:
        mw_times = np.array([i.time for i in curr_trialevents])
        
        # Get ID of each stimulus:
        mw_codes = []
        for i in curr_trialevents:
            if len(i.value)>2: # contains image stim
                if stimtype=='grating':
                    stim_config = (i.value[1]['rotation'], round(i.value[1]['frequency'],1))
                    stim_idx = [gidx for gidx,grating in enumerate(sorted(image_ids)) if grating==stim_config][0]+1
                else: # static image
                    #TODO:
                    # get appropriate stim_idx that is the equivalent of unique stim identity for images
                    pass
            else:
                stim_idx = 0
            mw_codes.append(stim_idx)
        mw_codes = np.array(mw_codes)
    
        # Append list of stimulus times and IDs for each SI file:
        mw_times_by_file.append(mw_times)
        mw_codes_by_file.append(mw_codes)
        mw_trials_by_file.append(curr_trialevents)
        
        # Calculate offset between first stimulus-update event and first SI-frame trigger:
        curr_offset = mw_times[0] - triggers[0] # time diff b/w detected frame trigger and stim display
        offsets_by_file.append(curr_offset)
        
        # Make silly dict() to keep organization consistent between event-based experiments, 
        # where each SI file contains multple discrete trials, versus movie-stimuli experiments, 
        # where each SI file contains one "trial" for that movie condition:
        if stimtype != 'bar':
            rkey = 'run'+str(idx)
            runs[rkey] = idx #dict() #i
        
    print "Average offset between stim update event and frame triggger is: ~%0.2f ms" % float(np.mean(offsets_by_file)/1000.)

mw_file_durs = [i[1]-i[0] for i in trigger_times]


Average offset between stim update event and frame triggger is: ~24.92 ms


In [17]:
# Rearrange dicts to match retino structures:
pydict = dict()

# Create "pydict" to store all MW stimulus/trial info in matlab-accessible format for GUI:
if stimtype=='bar':
    print "Offset between first MW stimulus-display-update event and first SI frame-trigger:"
    for ridx,run in enumerate(stimevents.keys()):
        pydict[run] ={'time': [i[0] for i in stimevents[run].states],\
                    'pos': stimevents[run].vals,\
                    'idxs': stimevents[run].idxs,\
                    'ordernum': stimevents[run].ordernum,\
                    'MWdur': (stimevents[run].states[-1][0] - stimevents[run].states[0][0]) / 1E6,\
                    'offset': stimevents[run].states[0][0] - stimevents[run].triggers[0],\
                    'MWtriggertimes': stimevents[run].triggers}
        print "run %i: %s ms" % (ridx+1, str(pydict[run]['offset']/1E3))
        
else:
    for ridx,run in enumerate(runs.keys()):
        pydict[run] = {'time': mw_times_by_file[ridx],\
                        'ordernum': runs[run], 
                        'offset': offsets_by_file[ridx],
                        'stimIDs': mw_codes_by_file[ridx],
                        'MWdur': mw_file_durs[ridx],\
                        'MWtriggertimes': trigger_times[ridx]}
        
        if stimtype=='grating':
            pydict[run]['idxs'] = [i for i,tmptev in enumerate(mw_trials_by_file[ridx]) if any(['gabor' in v['name'] for v in tmptev.value])],
        else:
            pydict[run]['idxs'] = [i for i,tmptev in enumerate(mw_trials_by_file[ridx]) if any([v['name'] in image_ids for v in tmptev.value])],


In [18]:
pydict.keys()

['run20',
 'run21',
 'run22',
 'run23',
 'run24',
 'run1',
 'run0',
 'run3',
 'run2',
 'run5',
 'run4',
 'run7',
 'run6',
 'run9',
 'run8',
 'run19',
 'run18',
 'run11',
 'run10',
 'run13',
 'run12',
 'run15',
 'run14',
 'run17',
 'run16']

In [19]:
# Ignoring ARDUINO stuff for now:
pydict['stimtype'] = stimtype

#pydict['mw_times_by_file'] = mw_times_by_file
#pydict['mw_file_durs'] = mw_file_durs
#pydict['mw_frame_trigger_times'] = frame_trigger_times
#pydict['offsets_by_file'] = offsets_by_file
#pydict['mw_codes_by_file'] = mw_codes_by_file
pydict['mw_dfn'] = mw_dfn
pydict['source_dir'] = source_dir
pydict['fn_base'] = fn_base
pydict['stimtype'] = stimtype
if stimtype=='bar':
    pydict['condtypes'] = ['left', 'right', 'top', 'bottom']
    pydict['runs'] = stimevents.keys()
    pydict['info'] = session_info
elif stimtype=='image':
    pydict['condtypes'] = sorted(image_ids)
    pydict['runs'] = runs.keys()
elif stimtype=='grating':
    pydict['condtypes'] = sorted(image_ids)
    pydict['runs'] = runs.keys()

tif_fn = fn_base+'.mat'
# scipy.io.savemat(os.path.join(source_dir, condition, tif_fn), mdict=pydict)
scipy.io.savemat(os.path.join(data_dir, 'mw_data', tif_fn), mdict=pydict)
print os.path.join(source_dir, 'mw_data', tif_fn)

    

/nas/volume1/2photon/RESDATA/mw_data/20161222_JR030W_gratings_20reps_run2.mat


In [20]:
pydict['condtypes']

[(0, 0.1),
 (0, 0.3),
 (0, 0.5),
 (0, 0.7),
 (0, 0.9),
 (15, 0.1),
 (15, 0.3),
 (15, 0.5),
 (15, 0.7),
 (15, 0.9),
 (30, 0.1),
 (30, 0.3),
 (30, 0.5),
 (30, 0.7),
 (30, 0.9),
 (45, 0.1),
 (45, 0.3),
 (45, 0.5),
 (45, 0.7),
 (45, 0.9),
 (60, 0.1),
 (60, 0.3),
 (60, 0.5),
 (60, 0.7),
 (60, 0.9),
 (75, 0.1),
 (75, 0.3),
 (75, 0.5),
 (75, 0.7),
 (75, 0.9),
 (90, 0.1),
 (90, 0.3),
 (90, 0.5),
 (90, 0.7),
 (90, 0.9)]

In [145]:
nupdates_per_stim = [len(tr) for tr in dynamic_stim_bitcodes]
p1 = figure(title="Stim ON durations (ms)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

#medintervals = np.array([i for i in intervals if i<=10000000])

hist, edges = np.histogram(np.array(nupdates_per_stim), density=True, bins=100)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

In [138]:
(ntrials*2 + ntrials) * 60

125820

In [89]:
import pymworks
df = None
df = pymworks.open(mw_dfns[0])
announce_trialend = df.get_events('Announce_TrialEnd')
announce_trialend = [t for t in announce_trialend if t.time<session_info['tboundary'][1] and t.time>session_info['tboundary'][0] and t.value==1]
trialend_evs = []
tidx= 0
for stim in sorted(stimevents, key=get_timekey):
    trialend_ev = next(t for t in sorted(announce_trialend[tidx:], key=get_timekey) if t.time>stim.time)
    tidx = [t for t in announce_trialend].index(trialend_ev)
    trialend_evs.append(announce_trialend[tidx])
    tidx += 1

print len(trialend_evs)

trial_durs = np.diff([t.time for t in sorted(announce_trialend, key=get_timekey)])
print min(trial_durs)
print max(trial_durs)


print len([t for t in trial_durs if t>3100000])

699
2006192
43949312
24


In [90]:
# From pymworks:
class Stimulus(object):
    def __init__(self, time, attributes, pixelclock=None):
        self.time = time
        self.attributes = attributes
        if pixelclock is not None:
            self.attributes['bit_code'] = pixelclock['bit_code']
        self.duration = None

    def __cmp__(self, other):
        return dict.__cmp__(self.attributes, other.attributes)

    def to_dict(self):
        d = self.attributes.copy()
        d.update(dict(time=self.time, duration=self.duration))
        return d
    
blacklisttests = [
    #lambda s: (('name' in s.keys()) and (s['name'] == 'pixel clock')),
    lambda s: (('type' in s.keys()) and (s['type'] == 'blankscreen')),
    ]

def to_stims(events, as_dicts=True, blacklist=None):
    if blacklist is None:
        blacklist = blacklisttests
    if not isinstance(blacklist, (tuple, list)):
        blacklist = (blacklist, )
    stims = []
    onscreen = []
    for e in sorted(events, key=lambda e: e.time):
        if e.value is None:
            logging.warning("Encountered event with value == None")
            if onscreen != {}:
                logging.error("Event.value == None with items on screen")
            continue
        current = []
        if hasattr(e.value, '__getitem__'):
            stimulus = None
            pixelclock = None
            for stim in e.value:
                if not isinstance(stim, dict) or \
                        any([t(stim) for t in blacklist]):
                    continue
                if ('name' in stim.keys()) and (stim['name'] == 'pixel clock'):
                    pixelclock = stim
                else:
                    if stimulus is not None:
                        logging.warning(
                            "Two stimuli onscreen: %s, %s"
                            % (stimulus, stim))
                    stimulus = stim
            if stimulus is not None:
                current.append(Stimulus(e.time, stimulus, pixelclock))
        #newstims, onscreen = find_stims(onscreen, current, e.time)
        stims += current #newstims
    if as_dicts:
        return [s.to_dict() for s in stims]
    return stims

def to_trials(stim_display_events, outcome_events, remove_unknown=True,
              duration_multiplier=2, stim_blacklists=None):
    """
    If remove_unknown, any trials where a corresponding outcome_event cannot
    be found will be removed.

    If duration_multiplier is not None, to_trials will check to see if the
    outcome event occured within duration_multiplier * duration microseconds
    of the trial start. If the outcome event occured later, the trial outcome
    will be marked as unknown.
    """
    if (len(outcome_events) == 0) or (len(stim_display_events) == 0):
        return []
    assert hasattr(outcome_events[0], 'name')

    trials = to_stims(stim_display_events, as_dicts=True,
                      blacklist=stim_blacklists)

    if (len(trials) == 0):
        return []

    outcomes = pymworks.events.utils.sync(outcome_events, trials,
                          direction=1, mkey=lambda x: x['time'])

    assert len(trials) == len(outcomes)
    unknowns = []
    if duration_multiplier is None:
        dtest = lambda t, o: True
    else:
        dtest = lambda t, o: \
            o.time < (t['time'] + t['duration'] * duration_multiplier)
    for i in xrange(len(trials)):
        if (outcomes[i] is not None) and dtest(trials[i], outcomes[i]):
            trials[i]['outcome'] = outcomes[i].name
            trials[i]['response_time'] = outcomes[i].time
        else:
            if remove_unknown:
                unknowns.append(i)
            else:
                trials[i]['outcome'] = 'unknown'

    # remove trials with 'unknown' outcome, in reverse
    for u in unknowns[::-1]:
        del trials[u]

    return trials

In [91]:
trials = to_trials(stimevents, trialend_evs,duration_multiplier=None)

In [106]:
dur = []
short = []
for idx,t in enumerate(sorted(trials, key=lambda e: e['start_time'])):
    dur.append(t['response_time'] - t['start_time'])
    if (t['response_time'] - t['start_time']) < 1000000:
        short.append(idx)

p1 = figure(title="Stim ON durations (ms)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

#medintervals = np.array([i for i in intervals if i<=10000000])

hist, edges = np.histogram(np.array(dur)/1E6, density=True, bins=100)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

In [109]:
np.diff(short)

array([ 8,  9,  6,  2,  4,  9,  3,  2,  1, 21, 16, 21,  4, 41, 32,  7,  8,
       80, 26,  9, 69,  3,  3,  3, 49,  9, 27,  4, 25, 31, 56, 42, 23])

In [114]:
bitcodes = [(i['start_time'], i['bit_code']) for i in trials]
#len([idx for idx,d in enumerate(np.diff(bitcodes)) if d==0])

In [116]:
display_updates = np.diff([b[0] for b in bitcodes])

p1 = figure(title="Stim ON durations (ms)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")
hist, edges = np.histogram(np.array(display_updates)/1E6, density=True, bins=100)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

In [122]:
new_file_idxs = [i+1 for i,dur in enumerate(display_updates) if dur>3100000]

In [123]:
new_file_idxs

[28,
 56,
 84,
 112,
 140,
 168,
 196,
 224,
 252,
 280,
 308,
 336,
 364,
 392,
 420,
 448,
 476,
 504,
 532,
 560,
 588,
 616,
 644,
 672]